# 共通処理

In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
import json

try:
    with open('setting.json', 'r') as file:
        setting = json.load(file)
except FileNotFoundError:
    print("先にset_run.ipynbで解析runを設定してください")

hdf5_path = setting['hdf5_path']
hdf5_path

In [ ]:
from model.File.HDF5 import HDF5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

# データを取り出す準備

In [ ]:
# hdf5内のpath
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    to_nxs = hdf.search_data_path(query='nxs_path')
    to_poni = hdf.search_data_path(query='poni_path')
    to_mask = hdf.search_data_path(query='mask_path')

In [ ]:
# ファイルのpathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    nxs_path = hdf.return_data(data_path=to_nxs)
    poni_path = hdf.return_data(data_path=to_poni)
    mask_path = hdf.return_data(data_path=to_mask)

In [ ]:
# nxsファイルとponiファイルを読み込む
from model.XRD.Nxs import NxsFile
nxs = NxsFile(nxs_path=nxs_path, poni_path=poni_path, mask_path=mask_path)

# データを取り出す

In [ ]:
# nxsファイルを開く
nxs.read_file()

In [ ]:
# hdfとしても開く
hdf_nxs = HDF5(f = h5py.File(nxs_path, 'r'))

In [ ]:
# 格納するときのlayerを作っておく
nxs_layer = 'entry/nxs/'

## 直接取り出す

In [ ]:
# nxsから直接取り出す変数
nxs_var = [
    'file_name',
    'frame_num',
    'count_time',
    'data',
    'saturation_value',
    'x_pixel_size',
    'y_pixel_size'
]

In [ ]:
# まずデータまでのpath
for var in nxs_var:
    exec(f"to_{var} = hdf_nxs.search_data_path(query=\"{var}\")") # to_~という変数でpathを取り出す

In [ ]:
# データを取り出す
for var in nxs_var:
    exec(f"{var} = hdf_nxs.return_data(data_path=to_{var})")
    exec(f"if type({var}[0]) == bytes:\n" + 
         f"\t{var} = {var}[0].decode('utf8')\n" + 
         f"elif {var}.shape == (1,):\n" +
         f"\t{var} = {var}[0]\n" +
         f"else:\n" + 
         f"\tpass")

In [ ]:
# 一応テスト
assert file_name.endswith('00000')
assert type(frame_num) == np.int32
assert type(count_time) == np.float64
assert type(data) == np.ndarray
assert type(saturation_value) == np.uint32
assert type(x_pixel_size) == np.float64
assert type(y_pixel_size) == np.float64

print("OK")

In [ ]:
# 一旦書き込む
with h5py.File(hdf5_path, 'a') as f_append:
    for var in nxs_var:
        exec(f"layer = nxs_layer + \"{var}\"")
        if var == 'data':        
            # exec(f"f_append.create_dataset(name=layer, data={var}, compression='gzip')") # データを節約したい場合はこちらを使う
            exec(f"f_append.create_dataset(name=layer, data={var})")
        else:
            exec(f"f_append.create_dataset(name=layer, data={var})")

In [ ]:
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

## 計算が必要なデータを作る

In [ ]:
# 加工して格納する変数
integrate_var = [
    'exposure_ms',
    'fps',
    'x_shape',
    'y_shape',
    'I',
    'cake'
]

In [ ]:
# 簡単な加工
exposure_ms = count_time / 1_000
print(f"{exposure_ms = }")

fps = 1 / exposure_ms
print(f"{fps = }")

x_shape = data.shape[1]
print(f"{x_shape = }")

y_shape = data.shape[2]
print(f"{y_shape = }")

del data
time.sleep(3)

In [ ]:
# 一旦書き込む
with h5py.File(hdf5_path, 'a') as f_append:
    for var in integrate_var[:4]:
        exec(f"layer = nxs_layer + \"{var}\"")
        exec(f"f_append.create_dataset(name=layer, data={var})")
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

In [ ]:
# 重たい加工
npt_rad = 2000
npt_azim = 360

In [ ]:
# 重たい加工 1: 1d_pattern
# 書き込んで消すまでやる

I = np.zeros((frame_num, npt_rad))

with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    to_data = hdf.search_data_path(query='nxs/data')
    
    # 積算する
    for frame in tqdm(range(frame_num)):
        data_tmp = hdf.return_data(data_path=to_data, shape=[frame])
        tth, I_tmp = nxs.ai.integrate1d(data=data_tmp,
                                        npt=npt_rad,
                                        unit="2th_deg",
                                        mask=nxs.mask)
        I[frame] = I_tmp

# 書き込み
with h5py.File(hdf5_path, 'a') as f_append:
    layer = nxs_layer + 'I'
    # f_append.create_dataset(name=layer, data=I, compression='gzip') # データを節約したい場合はこちらを使う
    f_append.create_dataset(name=layer, data=I)
    layer = nxs_layer + 'tth'
    f_append.create_dataset(name=layer, data=tth)

# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

time.sleep(3)
del I
time.sleep(3)


In [ ]:
# plt.imshow(I, aspect='auto') # 確認したい場合

In [ ]:
# 重たい加工 2: caking
# 書き込んで消すまでやる

cake = np.zeros((frame_num, npt_azim, npt_rad))

with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)
    to_data = hdf.search_data_path(query='nxs/data')

    # cakingする
    for frame in tqdm(range(frame_num)):
        data_tmp = hdf.return_data(data_path=to_data, shape=[frame])
        cake_tmp, tth, azi = nxs.ai.integrate2d(data=data_tmp,
                                                npt_rad=npt_rad,
                                                npt_azim=npt_azim,
                                                unit="2th_deg",
                                                mask=nxs.mask)
        cake[frame] = cake_tmp

time.sleep(3)
# 書き込み
print("書き込み開始")
with h5py.File(hdf5_path, 'a') as f_append:
    layer = nxs_layer + 'cake'
    # f_append.create_dataset(name=layer, data=cake, compression='gzip') # 保存のデータ容量を節約したい場合はこちらを使う
    f_append.create_dataset(name=layer, data=cake)
    layer = nxs_layer + 'azi'
    f_append.create_dataset(name=layer, data=azi)
print("書き込み終了")

time.sleep(3)
# 確認
with h5py.File(hdf5_path, 'r') as f_read:
    HDF5(f=f_read).show_all_hierarchy(f=f_read, display_length=80)

time.sleep(3)
del cake


# ファイルを閉じる

In [ ]:
want_to_close = False
if want_to_close:
    f.close()